<h1><center>Laboratorio 9:  ¿Superhéroe o Villano?  🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Alvaro Morales
- Nombre de alumno 2: Eduardo Faundez

### **Link de repositorio de GitHub:** `https://github.com/AlvaroMoralesS/Lab_Faundez_Morales`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [123]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
#!pip install umap-learn

# Librería para NLP
#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize  
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [124]:
# Si usted está utilizando Colaboratory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [125]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [126]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [127]:
df_comics.describe()

,Unnamed: 0,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
count,1285.000000,1285.000000,1285.000000,1285.000000,1285.000000,1285.000000,1285.000000,1228.000000,1228.000000,1228.000000,...,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000
mean,727.744747,78.665370,38.832685,45.898833,55.610895,66.556420,67.805447,0.089577,0.074919,0.074104,...,0.265472,0.307003,0.403909,0.429153,0.437296,0.387622,0.504886,0.540717,0.566775,0.522801
std,415.297480,25.074439,34.220845,29.176614,31.683003,33.329484,29.598315,0.285690,0.263367,0.262047,...,0.441764,0.461439,0.490880,0.495157,0.496255,0.487406,0.500180,0.498542,0.495723,0.499683
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,372.000000,75.000000,10.000000,25.000000,30.000000,40.000000,55.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,728.000000,85.000000,30.000000,40.000000,55.000000,70.000000,75.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
75%,1084.000000,95.000000,65.000000,65.000000,85.000000,100.000000,90.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1449.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [128]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [129]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [130]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [131]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [132]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)
pd.DataFrame(df.toarray(), columns=bow.get_feature_names())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [133]:
text_preprocessing = Pipeline([('Vect', CountVectorizer(tokenizer= StemmerTokenizer()))])
feature =['intelligence_score','strength_score','speed_score','durability_score','power_score','combat_score']

preprocess = ColumnTransformer([('text_preprocessing', text_preprocessing, 'history_text'),
                                ('MinMax'            ,MinMaxScaler(),      feature),
                                ])
pipeline = Pipeline([('preprocess', preprocess)])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [134]:
model = Pipeline([
    ('Codificador', pipeline),
    ('Selector'   , SelectPercentile(score_func=f_classif, percentile=90)),
    ('bayes'      , MultinomialNB()),
])

In [135]:
# División de muestra en train y test
X = df_comics.drop(columns=['alignment'])
y = LabelEncoder().fit_transform(df_comics['alignment'])
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.1, random_state=17615964)

In [136]:
#Entrenamiento
model.fit(X_train,y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.65      0.61      0.63        46
           1       0.72      0.84      0.78        70
           2       0.50      0.15      0.24        13

    accuracy                           0.69       129
   macro avg       0.62      0.54      0.55       129
weighted avg       0.67      0.69      0.67       129



In [137]:
model2 = Pipeline([
    ('Codificador', pipeline),
    ('Selector'   , SelectPercentile(score_func=f_classif, percentile=90)),
    ('DummyClass' , DummyClassifier(strategy='stratified')),
])

In [138]:
#Entrenamiento
model2.fit(X_train,y_train)
y_test_pred2 = model2.predict(X_test)
print(classification_report(y_test, y_test_pred2))

              precision    recall  f1-score   support

           0       0.34      0.30      0.32        46
           1       0.53      0.54      0.54        70
           2       0.00      0.00      0.00        13

    accuracy                           0.40       129
   macro avg       0.29      0.28      0.29       129
weighted avg       0.41      0.40      0.41       129



```
- El modelo MultinomialNB presenta mejores resultados que el modelo Dummy, dado que presenta un menor accuracy global. Analizando los valores de precision por categoría, en todas las clases el modelo entrenado presenta proporción de predicciones positivas.

- Si bien el modelo entrenado presenta resultados aceptables, sería interesante evaluar otros modelos de clasificación que puedan tener mejor capacidad de predicción.
```

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [139]:
model3 = Pipeline([
    ('Codificador', pipeline),
    ('Selector'   , SelectPercentile(score_func=f_classif, percentile=90)),
    ('classifier' , BaggingClassifier(random_state=42)),
])
#Entrenamiento
model3.fit(X_train,y_train)
y_test_pred2 = model3.predict(X_test)
print(classification_report(y_test, y_test_pred2))

              precision    recall  f1-score   support

           0       0.56      0.39      0.46        46
           1       0.62      0.86      0.72        70
           2       0.00      0.00      0.00        13

    accuracy                           0.60       129
   macro avg       0.39      0.42      0.39       129
weighted avg       0.54      0.60      0.55       129



c:\Users\alvaro.morales\AppData\Local\Continuum\anaconda3\envs\sskajs\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\alvaro.morales\AppData\Local\Continuum\anaconda3\envs\sskajs\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\alvaro.morales\AppData\Local\Continuum\anaconda3\envs\sskajs\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [140]:
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('text_preprocessing',
                                                  Pipeline(steps=[('Vect',
                                                                   CountVectorizer(tokenizer=<__main__.StemmerTokenizer object at 0x000001CB072068D0>))]),
                                                  'history_text'),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score'])]))])>

In [141]:
model2 = Pipeline([
    ('Codificador', pipeline),
    ('Selector'   , SelectPercentile(score_func=f_classif, percentile=90)),
    ('classifier' , RandomForestClassifier(random_state=42)),
])

params = [{'classifier': [RandomForestClassifier()],'classifier__n_estimators': [50, 100, 250], 'classifier__random_state':[42],
           'Selector':   [SelectPercentile()],      'Selector__percentile':     [70, 80, 90],
           'Codificador__preprocess__text_preprocessing__Vect':[CountVectorizer()],
           'Codificador__preprocess__text_preprocessing__Vect__ngram_range': [(1,1),(1,2),(1,3)], 
           'Codificador__preprocess__text_preprocessing__Vect__tokenizer': [StemmerTokenizer()]},
           
          {'classifier': [SVC()],                   'classifier__kernel':       ['linear','poly','rbf'],
           'Codificador__preprocess__text_preprocessing__Vect':[CountVectorizer()],
           'Codificador__preprocess__text_preprocessing__Vect__ngram_range': [(1,1),(1,2),(1,3)], 
           'Codificador__preprocess__text_preprocessing__Vect__tokenizer': [StemmerTokenizer()]},

          {'classifier': [BaggingClassifier()],     'classifier__n_estimators': [10,20,30],'classifier__random_state':[42],
           'Codificador__preprocess__text_preprocessing__Vect':[CountVectorizer()],
           'Codificador__preprocess__text_preprocessing__Vect__ngram_range': [(1,1),(1,2),(1,3)], 
           'Codificador__preprocess__text_preprocessing__Vect__tokenizer': [StemmerTokenizer()]},
          ]

In [142]:
cv = HalvingGridSearchCV(model2, params, verbose=10)
cv.fit(X_train,y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 42
max_resources_: 1156
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 45
n_resources: 42
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV 1/5; 1/45] START Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 1), Codificador__preprocess__text_preprocessing__Vect__tokenizer=<__main__.StemmerTokenizer object at 0x000001CB0875FC18>, Selector=SelectPercentile(), Selector__percentile=70, classifier=RandomForestClassifier(), classifier__n_estimators=50, classifier__random_state=42
[CV 1/5; 1/45] END Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 1), Codificador__preprocess__text_preprocessing__Vect__tokenizer=<__main__.StemmerTokenizer object at 0x000001CB0875FC18>, Selector=SelectPercentile(), Selec

c:\Users\alvaro.morales\AppData\Local\Continuum\anaconda3\envs\sskajs\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning:

divide by zero encountered in true_divide



[CV 3/5; 1/45] END Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 1), Codificador__preprocess__text_preprocessing__Vect__tokenizer=<__main__.StemmerTokenizer object at 0x000001CB0875FC18>, Selector=SelectPercentile(), Selector__percentile=70, classifier=RandomForestClassifier(), classifier__n_estimators=50, classifier__random_state=42;, score=(train=1.000, test=0.750) total time=   0.8s
[CV 4/5; 1/45] START Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 1), Codificador__preprocess__text_preprocessing__Vect__tokenizer=<__main__.StemmerTokenizer object at 0x000001CB0875FC18>, Selector=SelectPercentile(), Selector__percentile=70, classifier=RandomForestClassifier(), classifier__n_estimators=50, classifier__random_state=42


c:\Users\alvaro.morales\AppData\Local\Continuum\anaconda3\envs\sskajs\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning:

divide by zero encountered in true_divide



[CV 4/5; 1/45] END Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 1), Codificador__preprocess__text_preprocessing__Vect__tokenizer=<__main__.StemmerTokenizer object at 0x000001CB0875FC18>, Selector=SelectPercentile(), Selector__percentile=70, classifier=RandomForestClassifier(), classifier__n_estimators=50, classifier__random_state=42;, score=(train=1.000, test=0.625) total time=   0.4s
[CV 5/5; 1/45] START Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 1), Codificador__preprocess__text_preprocessing__Vect__tokenizer=<__main__.StemmerTokenizer object at 0x000001CB0875FC18>, Selector=SelectPercentile(), Selector__percentile=70, classifier=RandomForestClassifier(), classifier__n_estimators=50, classifier__random_state=42
[CV 5/5; 1/45] END Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codif

...

c:\Users\alvaro.morales\AppData\Local\Continuum\anaconda3\envs\sskajs\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning:

divide by zero encountered in true_divide



[CV 3/5; 45/45] END Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 3), Codificador__preprocess__text_preprocessing__Vect__tokenizer=<__main__.StemmerTokenizer object at 0x000001CB048C0C88>, classifier=BaggingClassifier(), classifier__n_estimators=30, classifier__random_state=42;, score=(train=1.000, test=0.625) total time=   1.3s
[CV 4/5; 45/45] START Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 3), Codificador__preprocess__text_preprocessing__Vect__tokenizer=<__main__.StemmerTokenizer object at 0x000001CB048C0C88>, classifier=BaggingClassifier(), classifier__n_estimators=30, classifier__random_state=42


c:\Users\alvaro.morales\AppData\Local\Continuum\anaconda3\envs\sskajs\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning:

divide by zero encountered in true_divide



[CV 4/5; 45/45] END Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 3), Codificador__preprocess__text_preprocessing__Vect__tokenizer=<__main__.StemmerTokenizer object at 0x000001CB048C0C88>, classifier=BaggingClassifier(), classifier__n_estimators=30, classifier__random_state=42;, score=(train=1.000, test=0.625) total time=   0.8s
[CV 5/5; 45/45] START Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 3), Codificador__preprocess__text_preprocessing__Vect__tokenizer=<__main__.StemmerTokenizer object at 0x000001CB048C0C88>, classifier=BaggingClassifier(), classifier__n_estimators=30, classifier__random_state=42
[CV 5/5; 45/45] END Codificador__preprocess__text_preprocessing__Vect=CountVectorizer(), Codificador__preprocess__text_preprocessing__Vect__ngram_range=(1, 3), Codificador__preprocess__text_preprocessing__Vect

HalvingGridSearchCV(estimator=Pipeline(steps=[('Codificador',
                                               Pipeline(steps=[('preprocess',
                                                                ColumnTransformer(transformers=[('text_preprocessing',
                                                                                                 Pipeline(steps=[('Vect',
                                                                                                                  CountVectorizer(tokenizer=<__main__.StemmerTokenizer object at 0x000001CB072068D0>))]),
                                                                                                 'history_text'),
                                                                                                ('MinMax',
                                                                                                 MinMaxScaler(),
                                                                                   

In [143]:
cv.best_params_

{'Codificador__preprocess__text_preprocessing__Vect': CountVectorizer(ngram_range=(1, 3),
                 tokenizer=<__main__.StemmerTokenizer object at 0x000001CB048C0C88>),
 'Codificador__preprocess__text_preprocessing__Vect__ngram_range': (1, 3),
 'Codificador__preprocess__text_preprocessing__Vect__tokenizer': <__main__.StemmerTokenizer at 0x1cb048c0c88>,
 'classifier': BaggingClassifier(n_estimators=20, random_state=42),
 'classifier__n_estimators': 20,
 'classifier__random_state': 42}

In [144]:
cv.best_score_

0.6392655257104986

```
- Dada la estructura de los datos, es decir, una cantidad de observaciones menor a 100.000, con predicción de tipo categórica y con datos de etiqueta, se consideró utilizar los modelos de Linear SVC, Bagging y Random Forest.

- Mediante la optimización de hiperparámetros, la mejor combinación se obtuvo mediante clasificador de Bagging con 20 estimadores y un codificador de rango (1,3), obteniendo un score de 0.64.
```

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [145]:
df_comics_nl = df_comics_no_label.dropna(subset=['history_text'])
X_test = df_comics_nl.drop(columns=['alignment'])

In [146]:
prediccion = cv.predict(X_test)

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>